In [1]:
import os

import pandas as pd

In [2]:
movies = pd.read_csv('data/cleaned-2/movies.csv').drop(columns='Unnamed: 0')
metadata = pd.read_csv('data/cleaned-2/metadata.csv').drop(columns='Unnamed: 0')

In [3]:
from langchain.docstore.document import Document

In [4]:
metadata['soup'] = movies['soup']

In [5]:
metadata.columns

Index(['belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'popularity',
       'poster_path', 'revenue', 'runtime', 'vote_average', 'vote_count',
       'spoken_languages', 'original_title', 'id', 'release_date', 'cast',
       'crew', 'genres', 'origin_country', 'original_language', 'overview',
       'production_companies', 'tagline', 'title', 'keywords', 'soup'],
      dtype='object')

In [6]:
movies = []

for idx, row in metadata.iterrows():
    x = Document(page_content=row['soup'],
                 metadata = {
                     "movie": row['title'],
                     "tmdb_id": row['id'],
                     "imdb_id": row['imdb_id'],
                     'genres': row['genres'],
                     "release_date": row['release_date'],
                     "cast": row['cast'],
                     "crew": row['crew'],
                     "collection": row['belongs_to_collection'],
                     "budget": row['budget'],
                     "revenue": row['revenue'],
                     "runtime": row['runtime'],
                     "language": row['original_language'],
                     "popularity": row['popularity'],
                     "synopsis": row['overview'],
                     "poster_path": row['poster_path'],
                     "homepage": row['homepage']
                 })
    movies.append(x)

In [7]:
movies[0]

Document(metadata={'movie': 'Ariel', 'tmdb_id': 2, 'imdb_id': 'tt0094675', 'genres': 'Comedy, Drama, Romance, Crime', 'release_date': '1988-10-21 00:00:00', 'cast': 'Turo Pajala, Susanna Haavisto, Matti Pellonpää, Eetu Hilkamo, Erkki Pajala, Matti Jaaranen, Hannu Viholainen, Jorma Markkula, Tarja Keinänen, Eino Kuusela, Kauko Laalo, Jyrki Olsonen, Esko Nikkari, Marja Packalén, Mikko Remes, Tomi Salmela, Reijo Marin, Heikki Salomaa, Veikko Uusimäki, Esko Salminen, Hannu Kivisalo, Pekka Wilen, Heikki Anttila, Pentti Auer, Timo Harakka, Kari Helaseppä, Juuso Hirvikangas, Hanna Jokinen, Sakari Kuosmanen, Sami Lanki, Jouko Lumme, Mikko Lyytikäinen, Timo Markko, Merja Pulkkinen, Markku Rantala, Sirkka Rautiainen, Jaakko Talaskivi, Timo Toikka, Olli Varja', 'crew': 'Additional Music: Dmitri Shostakovich, Additional Music: Pyotr Ilyich Tchaikovsky, Assistant Camera: Heikki Ortamo, Assistant Camera: Timo Markko, Assistant Director: Pauli Pentti, Assistant Editor: Outi Hyytinen, Colorist: Pirjo 

In [8]:
len(movies)

11307

In [72]:
import weaviate
from utils.api_keys import fetch_api_key, add_api_key

In [145]:
add_api_key("weaviate2")

Key loading successful.
Modified existing key
Keys and password successfully saved to file.


In [146]:
# client = weaviate.Client(
#     url="https://cjscqn5qa6ddwpdf5ucna.c0.asia-southeast1.gcp.weaviate.cloud",
#     auth_client_secret=weaviate.AuthApiKey(fetch_api_key("weaviate", False))
# )
client = weaviate.Client(
    url="https://vz10aml8qwbzfzxohgk7a.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_client_secret=weaviate.AuthApiKey(fetch_api_key("weaviate2", False)))

C:\Users\Parag\anaconda3\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\Parag\AppData\Local\Temp\ipykernel_23152\2865237510.py:5: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(
C:\Users\Parag\anaco

Key loading successful.


In [147]:
from langchain_huggingface import HuggingFaceEmbeddings

In [148]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [149]:
from langchain.vectorstores import Weaviate

In [ ]:
# Weaviate()

In [150]:
# vector_db = Weaviate.from_documents(
#     movies,
#     embeddings,
#     client = client,
#     by_text = False
# )
vector_db = Weaviate(client=client,embedding=embeddings,
                      index_name="Mrsprj",
                      text_key="movies",
                      by_text=False,
                     attributes=list(movies[0].metadata.keys()),
                     )

In [151]:
# vector_db.add_documents(movies)

['72f34896-b285-4f30-bc28-a923a190e290',
 'df605bff-fc30-4c51-9371-cb4796909e2a',
 '36767f4a-6d47-4c2b-8691-4678b938ec32',
 '68ca3498-d31c-486c-8034-3f1585455070',
 '6252c68a-8358-451d-b24b-046901cfa791',
 'd5a1a528-f605-4b75-9e7e-f9d1554ed76f',
 '6ebfe3b1-0319-4ab1-9f3a-c88922e3dcd7',
 'a9da2fea-f9c2-4832-95e4-802b8db5c6ce',
 '0ddf4a3b-3709-4f31-99d3-a2f00abc01da',
 'd04f6251-dd7b-482b-bef7-26d06a6c33bb',
 '0eccc638-5dd7-4fe6-8479-9fef1278dfef',
 '99d3e7b4-c808-48ef-b2c9-909e780c11bd',
 '216bdfc9-d20f-449d-ad27-5b403f6bbd17',
 '55180b14-30e1-4210-ac6b-f3bb82d08d71',
 '9353fa84-b6af-4318-ac8d-26240ec76d43',
 '3f655a03-3b2f-4c73-a116-05f8f5f13eb9',
 '391be9de-4484-4e39-b028-a3626800bf30',
 '35d5a7ff-a1ca-44c0-b6f3-fe4bcb8e4a42',
 '4393298e-d8d0-486d-9a92-e72fb0bee253',
 'dd758861-80a9-4448-b95b-b29cbe29ac26',
 '1173867f-7e40-4599-bfc3-4062a3273316',
 '7e9073f4-1621-44f7-a334-7ec748eb3956',
 '02e4a809-07ed-4bfe-9d78-a6490b42b89b',
 'ab56fe0b-70df-46d2-97fb-ef3ce26323f8',
 '419a72b4-2cbe-

In [152]:
soups = pd.Series(metadata['soup'].values, index=metadata['title'])

In [153]:
soups

title
Ariel                                 Title: Ariel. Genres: Comedy Drama Romance Cri...
Shadows in Paradise                   Title: Shadows in Paradise. Genres: Comedy Dra...
Four Rooms                            Title: Four Rooms. Genres: Comedy. Keywords: h...
Judgment Night                        Title: Judgment Night. Genres: Action Crime Th...
Life in Loops (A Megacities RMX)      Title: Life in Loops (A Megacities RMX). Genre...
                                                            ...                        
Zero: An Investigation Into 9/11      Title: Zero: An Investigation Into 9/11. Genre...
The Concert                           Title: The Concert. Genres: Drama Comedy. Keyw...
The Caveman's Valentine               Title: The Caveman's Valentine. Genres: Drama ...
New World Disorder 9: Never Enough    Title: New World Disorder 9: Never Enough. Key...
Sesame Street: Elmo Loves You!        Title: Sesame Street: Elmo Loves You!. Genres:...
Length: 11307, dtype: obje

In [160]:
def get_recommendations(title, vector_db):

    if title not in soups:
        raise ValueError(f"Title '{title}' not found in indices")

    query = soups.get(title)

    try:
        results = vector_db.similarity_search_with_score(query, k=11)

        top_ten = []

        for x in results[1:]:

            movie_metadata = {
                'movie': x[0].metadata['movie'],
                "tmdb_id": x[0].metadata['tmdb_id'],
                "imdb_id": x[0].metadata['imdb_id'],
                'genres': x[0].metadata['genres'],
                "release_date": x[0].metadata['release_date'],
                "cast": x[0].metadata['cast'],
                "crew": x[0].metadata['crew'],
                "collection": x[0].metadata['collection'],
                "budget": x[0].metadata['budget'],
                "revenue": x[0].metadata['revenue'],
                "runtime": x[0].metadata['runtime'],
                "language": x[0].metadata['language'],
                "popularity": x[0].metadata['popularity'],
                "synopsis": x[0].metadata['synopsis'],
                "poster_path": x[0].metadata['poster_path'],
                "homepage": x[0].metadata['homepage'],
                'similarity_score': round(x[1], 2),
            }
            top_ten.append(movie_metadata)

        df_top_ten = pd.DataFrame(top_ten)
        return df_top_ten

    except Exception as e:
        print(f"Error during query: {e}")
        return None

In [161]:
metadata.title

0                                     Ariel
1                       Shadows in Paradise
2                                Four Rooms
3                            Judgment Night
4          Life in Loops (A Megacities RMX)
                        ...                
11302      Zero: An Investigation Into 9/11
11303                           The Concert
11304               The Caveman's Valentine
11305    New World Disorder 9: Never Enough
11306        Sesame Street: Elmo Loves You!
Name: title, Length: 11307, dtype: object

In [183]:
querry = metadata[metadata['title'].str.contains('Star Wars')]['title'].values[0]
rec = get_recommendations(querry, vector_db)

In [184]:
rec

,movie,tmdb_id,imdb_id,genres,release_date,cast,crew,collection,budget,revenue,runtime,language,popularity,synopsis,poster_path,homepage,similarity_score
0,The Empire Strikes Back,1891,tt0080684,"Adventure, Action, Science Fiction",1980-05-20 00:00:00,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...","Animation: Phil Tippett, Art Direction: Alan T...",Star Wars Collection,18000000,538400000,124,en,35.741,"The epic saga continues as Luke Skywalker, in ...",/nNAeTmF4CtdSgMDplXTDPOpYzsX.jpg,http://www.starwars.com/films/star-wars-episod...,0.73
1,Return of the Jedi,1892,tt0086190,"Adventure, Action, Science Fiction",1983-05-25 00:00:00,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...","Assistant Production Manager: Patricia Carr, C...",Star Wars Collection,32350000,572700000,132,en,38.062,Luke Skywalker leads a mission to rescue his f...,/jQYlydvHm3kUix1f8prMucrplhm.jpg,http://www.starwars.com/films/star-wars-episod...,0.73
2,Star Wars: Episode I - The Phantom Menace,1893,tt0120915,"Adventure, Action, Science Fiction",1999-05-19 00:00:00,"Liam Neeson, Ewan McGregor, Natalie Portman, J...","ADR Editor: Gwendolyn Yates Whittle, ADR Edito...",Star Wars Collection,115000000,924317558,136,en,40.345,"Anakin Skywalker, a young slave strong with th...",/6wkfovpn7Eq8dYNKaG5PY3q2oq6.jpg,http://www.starwars.com/films/star-wars-episod...,0.61
3,Star Wars: Episode II - Attack of the Clones,1894,tt0121765,"Adventure, Action, Science Fiction",2002-05-15 00:00:00,"Hayden Christensen, Ewan McGregor, Natalie Por...","ADR Editor: Marilyn McCoppen, Additional Sound...",Star Wars Collection,120000000,649398328,142,en,35.045,Following an assassination attempt on Senator ...,/oZNPzxqM2s5DyVWab09NTQScDQt.jpg,[MISSING],0.59
4,Starcrash,22049,tt0079946,"Fantasy, Science Fiction",1978-12-21 00:00:00,"Marjoe Gortner, Caroline Munro, Christopher Pl...","Additional Dialogue: R. A. Dillon, Animation: ...",Star Crash Collection,0,0,94,it,9.243,A pair of smugglers manage to pick up a castaw...,/npJt35Q36aGcTYpcg9epbwQIIkz.jpg,[MISSING],0.55
5,Star Wars: Episode III - Revenge of the Sith,1895,tt0121766,"Adventure, Action, Science Fiction",2005-05-17 00:00:00,"Hayden Christensen, Ewan McGregor, Natalie Por...","ADR Editor: Jeremy Molod, ADR Editor: Steve Sl...",Star Wars Collection,113000000,850000000,140,en,38.914,The evil Darth Sidious enacts his final plan f...,/xfSAoBEm9MNBjmlNcDYLvLSMlnq.jpg,http://www.starwars.com/films/star-wars-episod...,0.54
6,Spaceballs,957,tt0094012,"Comedy, Science Fiction",1987-06-24 00:00:00,"Mel Brooks, John Candy, Rick Moranis, Bill Pul...","Additional Editing: Nicholas C. Smith, Art Dir...",[MISSING],22700000,38119483,96,en,26.105,When the nefarious Dark Helmet hatches a plan ...,/aHNeKtkNnyQcietzi2cgiYL8sC9.jpg,http://www.mgm.com/view/movie/1873/Spaceballs/,0.50
7,Serenity,16320,tt0379786,"Science Fiction, Action, Adventure, Thriller",2005-09-25 00:00:00,"Nathan Fillion, Summer Glau, Gina Torres, Alan...","3D Artist: Mary E. Manning, ADR Editor: Stepha...",[MISSING],39000000,38869464,119,en,30.552,When the renegade crew of Serenity agrees to h...,/4sqUOaPFoP2W81mq1UYqZqf5WzA.jpg,[MISSING],0.50
8,Alien Agent,19279,tt0820466,"Action, Science Fiction",2007-07-17 00:00:00,"Mark Dacascos, Amelia Cooke, Emma Lahana, Kim ...","ADR Supervisor: Chris Ove, Aerial Camera: A.J....",[MISSING],0,0,95,en,3.113,A lawman from another galaxy must stop an inva...,/vdgI2z1waA5NFVtv373AhzPOKaF.jpg,[MISSING],0.48
9,Battle Planet,17571,tt1016024,"Action, Science Fiction",2008-11-02 00:00:00,"Zack Ward, Monica May, Colleen Smith, Kevin Th...","Art Direction: Kevin Ivers, Assistant Makeup A...",[MISSING],0,0,85,en,2.373,"In the not-so-distant future, Captain Jordan S...",/bAOhiu2zS1KEEEPBEg1fnUydfrd.jpg,http://www.battleplanetmovie.com/,0.48


In [185]:
rec.columns

Index(['movie', 'tmdb_id', 'imdb_id', 'genres', 'release_date', 'cast', 'crew',
       'collection', 'budget', 'revenue', 'runtime', 'language', 'popularity',
       'synopsis', 'poster_path', 'homepage', 'similarity_score'],
      dtype='object')

In [186]:
metadata[metadata['title'].str.contains("Vampire")]['title'].values[0]

'Interview with the Vampire'

In [188]:
metadata['id']

0            2
1            3
2            5
3            6
4            8
         ...  
11302    25204
11303    25205
11304    25208
11305    25449
11306    31975
Name: id, Length: 11307, dtype: int64

In [189]:
metadata['soup']

0        Title: Ariel. Genres: Comedy Drama Romance Cri...
1        Title: Shadows in Paradise. Genres: Comedy Dra...
2        Title: Four Rooms. Genres: Comedy. Keywords: h...
3        Title: Judgment Night. Genres: Action Crime Th...
4        Title: Life in Loops (A Megacities RMX). Genre...
                               ...                        
11302    Title: Zero: An Investigation Into 9/11. Genre...
11303    Title: The Concert. Genres: Drama Comedy. Keyw...
11304    Title: The Caveman's Valentine. Genres: Drama ...
11305    Title: New World Disorder 9: Never Enough. Key...
11306    Title: Sesame Street: Elmo Loves You!. Genres:...
Name: soup, Length: 11307, dtype: object

In [190]:
metadata

,belongs_to_collection,budget,homepage,imdb_id,popularity,poster_path,revenue,runtime,vote_average,vote_count,...,crew,genres,origin_country,original_language,overview,production_companies,tagline,title,keywords,soup
0,Kaurismäki's Proletariat Trilogy,0,[MISSING],tt0094675,11.403,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,0,73,7.100,336,...,"Additional Music: Dmitri Shostakovich, Additio...","Comedy, Drama, Romance, Crime",FI,fi,After the coal mine he works at closes and his...,Villealfa Filmproductions,[MISSING],Ariel,"prison, underdog, helsinki, finland, factory w...",Title: Ariel. Genres: Comedy Drama Romance Cri...
1,Kaurismäki's Proletariat Trilogy,0,[MISSING],tt0092149,7.066,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,0,74,7.297,399,...,"Assistant Camera: Heikki Ortamo, Assistant Cam...","Comedy, Drama, Romance",FI,fi,"Nikander, a rubbish collector and would-be ent...",Villealfa Filmproductions,[MISSING],Shadows in Paradise,"helsinki, finland, salesclerk, garbage",Title: Shadows in Paradise. Genres: Comedy Dra...
2,[MISSING],4000000,https://www.miramax.com/movie/four-rooms/,tt0113101,15.843,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,4257354,98,5.853,2639,...,"ADR Editor: Zack Davis, ADR Mixer: Christina T...",Comedy,US,en,It's Ted the Bellhop's first night on the job....,"Miramax, A Band Apart",Twelve outrageous guests. Four scandalous requ...,Four Rooms,"hotel, new year's eve, witch, bet, sperm, hote...",Title: Four Rooms. Genres: Comedy. Keywords: h...
3,[MISSING],21000000,[MISSING],tt0107286,12.929,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,12136938,109,6.500,331,...,"Art Direction: Dan Olexiewicz, Associate Produ...","Action, Crime, Thriller",US,en,"Four young friends, while taking a shortcut en...","Largo Entertainment, JVC, Universal Pictures",Don't move. Don't whisper. Don't even breathe.,Judgment Night,"drug dealer, chicago, illinois, escape, one ni...",Title: Judgment Night. Genres: Action Crime Th...
4,[MISSING],42000,http://lifeinloops.com,tt0825671,4.073,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg,0,80,7.300,28,...,"Director of Photography: Wolfgang Thaler, Dire...",Documentary,AT,en,Timo Novotny labels his new project an experim...,inLoops,A Megacities remix.,Life in Loops (A Megacities RMX),megacities,Title: Life in Loops (A Megacities RMX). Genre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11302,[MISSING],0,[MISSING],tt1297858,1.733,/rZW8WXGyNnzuuPlswcdINVXkBp3.jpg,0,104,6.100,18,...,"Cinematography: Christian Di Prinzio, Cinemato...",Documentary,IT,it,ZERO: An Investigation into 9/11 has one centr...,"Telemaco, TPF Telemaco, theGLAV",[MISSING],Zero: An Investigation Into 9/11,"9/11, twin towers",Title: Zero: An Investigation Into 9/11. Genre...
11303,[MISSING],0,[MISSING],tt1320082,7.251,/zQHwJk6t6AUBjrQCj2ksiShRy2o.jpg,0,119,7.247,497,...,"Adaptation: Radu Mihăileanu, Art Direction: Ar...","Drama, Comedy",FR,fr,A former world-famous conductor of the Bolshoï...,"Wild Bunch, Oï Oï Oï Productions, Les Produc...",The grand deception that became the performanc...,The Concert,"concert, orchestra, musical, violin player, gipsy",Title: The Concert. Genres: Drama Comedy. Keyw...
11304,[MISSING],13500000,[MISSING],tt0182000,7.220,/xWjoiGvFZPXuBi93reGblvDHIax.jpg,794481,105,5.729,133,...,"Art Direction: Grant Van Der Slagt, Co-Executi...","Drama, Mystery, Thriller",US,en,"Romulus, a misunderstood musician turned reclu...","Jersey Shore, Franchise Pictures",Romulus Ledbetter lives on the edge. The view ...,The Caveman's Valentine,woman director,Title: The Caveman's Valentine. Genres: Drama ...
11305,New World Disorder,0,http://www.nwdfilms.com,[MISSING],2.102,/itQjoIO2e3padeLliHROJVAb7Av.jpg,0,69,6.000,3,...,Director: Derek Westerlund,[MISSING],US,en,Gee Atherton ripping the Worlds course the day...,[MISSING],[MISSING],New World Disorder 9: Never Enough,"sports, mountain biking",Title: New World Disorder 9: Never Enough. Key...


In [ ]:
def get_recommendations(title, vector_db):

    if title not in soups:
        raise ValueError(f"Title '{title}' not found in indices")

    query = soups.get(title)

    try:
        results = vector_db.similarity_search_with_score(query, k=11)

        top_ten = []

        for x in results[1:]:

            movie_metadata = {
                'movie': x[0].metadata['movie'],
                "tmdb_id": x[0].metadata['tmdb_id'],
                "imdb_id": x[0].metadata['imdb_id'],
                'genres': x[0].metadata['genres'],
                "release_date": x[0].metadata['release_date'],
                "cast": x[0].metadata['cast'],
                "crew": x[0].metadata['crew'],
                "collection": x[0].metadata['collection'],
                "budget": x[0].metadata['budget'],
                "revenue": x[0].metadata['revenue'],
                "runtime": x[0].metadata['runtime'],
                "language": x[0].metadata['language'],
                "popularity": x[0].metadata['popularity'],
                "synopsis": x[0].metadata['synopsis'],
                "poster_path": x[0].metadata['poster_path'],
                "homepage": x[0].metadata['homepage'],
                'similarity_score': round(x[1], 2),
            }
            top_ten.append(movie_metadata)

        df_top_ten = pd.DataFrame(top_ten)
        return df_top_ten

    except Exception as e:
        print(f"Error during query: {e}")
        return None

In [271]:
import os

In [272]:
os.makedirs(name="data/final", exist_ok=True)
metadata.to_csv("data/final/final_metadata.csv")

In [292]:
class Recommender:
    def __init__(self, vectorstore, metadata_path):

        self.vectorstore = vectorstore
        self.metadata = pd.read_csv(metadata_path)

    @classmethod
    def from_weaviate(cls, metadata_path, embedding_model_args, weaviate_args ):
        client = weaviate.Client(
            url=weaviate_args['url'],
            auth_client_secret=weaviate.AuthApiKey(fetch_api_key(weaviate_args['ak_name'], False))
        )
        embeddings = HuggingFaceEmbeddings(
          model_name=embedding_model_args['model_name'],
          model_kwargs=embedding_model_args['model_kwargs']
        )
        vectorstore = Weaviate(client=client,embedding=embeddings,
                      index_name=weaviate_args['index_name'],
                      text_key=weaviate_args['text_key'],
                      by_text=weaviate_args['by_text'],
                     attributes=weaviate_args['attributes'],
                     )
        rec = cls(metadata_path = metadata_path, vectorstore=vectorstore)
        return rec

    def guess_movie(self, keyword):
        return self.metadata[self.metadata['title'].str.contains(keyword)]['title'].values[0]

    def get_recommendations_by_id(self, tmdb_id, k=10):
        if tmdb_id not in self.metadata.id:
            raise ValueError(f"Id '{tmdb_id}' not found in indices")
        querry = self.metadata[self.metadata['id'] == tmdb_id][['soup']].values[0]
        return self.recommend(querry[0], k)

    def get_recommendations_by_title(self, title, k=10):
        if title not in list(self.metadata.title):
            raise ValueError(f"title '{title}' not found in indices")
        querry = self.metadata[self.metadata['title'] == title][['soup']].values[0]
        return self.recommend(querry[0], k)

    def get_recommendations_by_keywords(self, keyword, k = 10):
        title = self.guess_movie(keyword)
        return self.get_recommendations_by_title(title, k = k)

    def recommend(self, query, k):
        try:
            results = self.vectorstore.similarity_search_with_score(query, k=k+1)

            top_k = []

            for x in results[1:]:

                movie_metadata = {
                    'movie': x[0].metadata['movie'],
                    "tmdb_id": x[0].metadata['tmdb_id'],
                    "imdb_id": x[0].metadata['imdb_id'],
                    'genres': x[0].metadata['genres'],
                    "release_date": x[0].metadata['release_date'],
                    "cast": x[0].metadata['cast'],
                    "crew": x[0].metadata['crew'],
                    "collection": x[0].metadata['collection'],
                    "budget": x[0].metadata['budget'],
                    "revenue": x[0].metadata['revenue'],
                    "runtime": x[0].metadata['runtime'],
                    "language": x[0].metadata['language'],
                    "popularity": x[0].metadata['popularity'],
                    "synopsis": x[0].metadata['synopsis'],
                    "poster_path": x[0].metadata['poster_path'],
                    "homepage": x[0].metadata['homepage'],
                    'similarity_score': round(x[1], 2),
                }
                top_k.append(movie_metadata)

            df_top_k = pd.DataFrame(top_k)
            return df_top_k

        except Exception as e:
            print(f"Error during query: {e}")
            return None


In [293]:
import yaml

In [294]:
def load_kwargs(fpath):
    with open(fpath, 'r') as file:
        config = yaml.safe_load(file)
    return config

In [298]:
rec = Recommender.from_weaviate(**load_kwargs("config/weaviate.yaml"))

C:\Users\Parag\anaconda3\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\Parag\AppData\Local\Temp\ipykernel_23152\3872837225.py:9: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(
C:\Users\Parag\anaco

Key loading successful.


In [299]:
rec.get_recommendations_by_keywords("Batman")

,movie,tmdb_id,imdb_id,genres,release_date,cast,crew,collection,budget,revenue,runtime,language,popularity,synopsis,poster_path,homepage,similarity_score
0,The Dark Knight,155,tt0468569,"Drama, Action, Crime, Thriller",2008-07-16 00:00:00,"Christian Bale, Heath Ledger, Aaron Eckhart, M...","""A"" Camera Operator: Bob Gorelick, ADR Supervi...",The Dark Knight Collection,185000000,1004558444,152,en,142.909,Batman raises the stakes in his war on crime. ...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,https://www.warnerbros.com/movies/dark-knight/,0.79
1,Batman Begins,272,tt0372784,"Action, Crime, Drama",2005-06-10 00:00:00,"Christian Bale, Michael Caine, Liam Neeson, Ka...","2D Supervisor: Charlie Noble, 3D Supervisor: D...",The Dark Knight Collection,150000000,374218673,140,en,54.696,"Driven by tragedy, billionaire Bruce Wayne ded...",/4MpN4kIEqUjW8OPtOQJXlTdHiJV.jpg,https://www.warnerbros.com/movies/batman-begins/,0.72
2,Batman Beyond: Return of the Joker,16234,tt0233298,"Animation, Family, Action, Science Fiction",2000-12-12 00:00:00,"Will Friedle, Kevin Conroy, Mark Hamill, Angie...","Director of Photography: Yoshihiro Kasahara, D...",Batman Beyond Collection,0,0,77,en,21.606,"The Joker is back with a vengeance, and Neo-Go...",/rsqoX8tA6vD8kvJSQwLN8Df55nk.jpg,[MISSING],0.71
3,Batman: Mystery of the Batwoman,21683,tt0346578,"Adventure, Fantasy, Animation, Action, Science...",2003-10-21 00:00:00,"Kevin Conroy, Kimberly Brooks, Kelly Ripa, Eli...","ADR Editor: Kelly Ann Foley, Associate Produce...",Batman (DC Universe Animated) Collection,0,0,74,en,17.572,As if the Penguin wasn't enough to contend wit...,/mlmhpUArJdpRPO211v3lETe3uzg.jpg,[MISSING],0.68
4,Batman & Robin,415,tt0118688,"Action, Science Fiction, Adventure",1997-06-20 00:00:00,"Arnold Schwarzenegger, George Clooney, Chris O...","3D Modeller: Domi Piturro, ADR Editor: Bobbi B...",Batman Collection,125000000,238207122,125,en,26.257,Batman and Robin deal with relationship issues...,/cGRDufDDSrFrv7VI4YnmWnslne0.jpg,[MISSING],0.66
5,The Batman/Superman Movie: World's Finest,17074,tt0169590,"Action, Animation, TV Movie, Science Fiction",1997-10-04 00:00:00,"Tim Daly, Kevin Conroy, Dana Delany, Clancy Br...","Additional Music: Lolita Ritmanis, Art Designe...",Superman (DC Universe Animated) Collection,0,0,64,en,9.883,As the Joker visits Metropolis with a plan to ...,/dQxwNGQ6pQLCUE5vaI9PwIZK6UC.jpg,[MISSING],0.65
6,Batman Forever,414,tt0112462,"Action, Crime, Fantasy",1995-06-16 00:00:00,"Val Kilmer, Tommy Lee Jones, Jim Carrey, Nicol...","ADR Editor: Zack Davis, ADR Mixer: Troy Porter...",Batman Collection,100000000,336529144,121,en,25.674,Batman must battle a disfigured district attor...,/mzzNBVwTiiY94xAXDMWJpNPW2US.jpg,[MISSING],0.64
7,Batman,2661,tt0060153,"Action, Comedy, Crime",1966-07-30 00:00:00,"Adam West, Burt Ward, Lee Meriwether, Cesar Ro...","Art Direction: Serge Krizman, Associate Produc...",[MISSING],1377800,3900000,105,en,15.902,The Dynamic Duo faces four super-villains who ...,/zzoPxWHnPa0eyfkMLgwbNvdEcVF.jpg,[MISSING],0.63
8,Darkman,9556,tt0099365,"Action, Science Fiction, Thriller",1990-08-24 00:00:00,"Liam Neeson, Frances McDormand, Colin Friels, ...","ADR Editor: Andrea Horta, ADR Editor: Joseph A...",Darkman Collection,16000000,48900000,95,en,14.725,Dr. Peyton Westlake is on the verge of realizi...,/9fxGRzlINIvfPFhizMgbQaDDrK.jpg,[MISSING],0.61
9,Black Dawn,18070,tt0443450,Action,2006-02-13 00:00:00,"Steven Seagal, Tamara Davies, John Pyper-Fergu...","Art Direction: Ni Ni Than, Associate Producer:...",The Foreigner Collection,8000000,63709,96,en,11.386,"Jonathan Cold returns, this time he goes Under...",/jUqyNMJQ45fRsIiIqU5TlcJnpif.jpg,[MISSING],0.61


In [300]:
metadata.columns

Index(['belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'popularity',
       'poster_path', 'revenue', 'runtime', 'vote_average', 'vote_count',
       'spoken_languages', 'original_title', 'id', 'release_date', 'cast',
       'crew', 'genres', 'origin_country', 'original_language', 'overview',
       'production_companies', 'tagline', 'title', 'keywords', 'soup'],
      dtype='object')